# 1D Finite Elements with Explicit Time Integration

## Simple Integrator

In this notebook we look to describe the process of solving a simple Wave Propagation example 
with Explicit Finite Elements in 1D

In [5]:
# First we import some libraries that will help us out 
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('..')
from BoundaryConditions import  VelBoundaryConditions as vbc
from BoundaryConditions import  AccelBoundaryConditions as abc
import imageio


## Create a Simple Integrator Class

We create a module for both the Total and Updated Lagrangian Formulation that integrates in time
It contains the following methods:
 - Assembly of Internal Forces including the time step computation
 - Assembly of Kinetic Boundary Conditions ($v, a$)
 - Time Integration Solver (Leapfrog)
 - Computation of Basic Energy Quantities
 

In [6]:
class SimpleIntegrator:
    """
    Constructor for the Simple integrator
    :param formulation: Total or Updated Lagrangian
    :param young:     Young´s modulus
    :param density:   Density
    :param length:     length of bar
    :param A:     Area of bar
    :param num_elems: #elements in bar
    :param tfinal: final time of computation
    :param Co: Courant number
    :param v_bc: velocity boundary condition
    """

    def __init__(self, formulation, young, density, length, A, num_elems, tfinal, v_bc: vbc, a_bc: abc, Co=1.0):
        self.formulation = formulation
        self.E = young
        self.rho = density
        self.L = length
        self.n_elem = num_elems
        self.tfinal = tfinal
        self.Co = Co
        self.n_nodes = num_elems + 1
        self.position, self.dx = np.linspace(0, length, self.n_nodes, retstep=True)
        self.midposition = [self.position[n] + 0.5 * self.dx for n in range(0, len(self.position)-1)]
        self.v_bc = v_bc
        self.a_bc = a_bc
        self.n = 0
        self.t = 0
        self.a = np.zeros(self.n_nodes)
        self.v = np.zeros(self.n_nodes)
        self.u = np.zeros(self.n_nodes)
        self.stress = np.zeros(self.n_elem)
        self.strain = np.zeros(self.n_elem)
        self.f_int = np.zeros(self.n_nodes)
        self.dt = Co * self.dx * np.sqrt(self.rho / self.E)
        nodalMass = 0.5 * self.rho * self.dx
        self.mass = np.ones(self.n_nodes) * nodalMass
        self.mass[1:-1] *= 2
        self.kinetic_energy = []
        self.internal_energy = []
        self.tot_energy = []
        self.timestamps = []

    def assemble_internal(self):
        if (self.formulation == "updated"):
            tempdx = [self.position[n]-self.position[n-1] for n in range(1, len(self.position))] # Updated Lagrangian
            self.midposition = [self.position[n] + 0.5 * tempdx[n] for n in range(0, len(self.position)-1)]
            self.dt = self.Co * min(tempdx) * np.sqrt(self.rho / self.E) # Updated Lagrangian
            self.strain = np.zeros(self.n_elem)
            self.strain = (np.diff(self.v) / tempdx) # Strain Measure is Rate of Deformation
            self.stress += self.strain * self.dt * self.E # Updated Lagrangian

            self.f_int[1:-1] = -np.diff(self.stress)
            self.f_int[0] += -self.stress[0]
            self.f_int[-1] += self.stress[-1]

        if (self.formulation == "total"):
            self.stress = np.zeros(self.n_elem)
            self.strain = np.zeros(self.n_elem)
            self.strain = (np.diff(self.u) / self.dx) 
            self.stress = self.strain * self.E

            self.f_int[1:-1] = -np.diff(self.stress)
            self.f_int[0] += -self.stress[0]
            self.f_int[-1] += self.stress[-1]

    def assemble_vbcs(self, t):
        if (self.v_bc):
            for counter in range(0, len(self.v_bc.indexes)):
                self.v[self.v_bc.indexes[counter]] = self.v_bc.velocities[counter](t)

    def assemble_abcs(self):
        if (self.a_bc):
            for counter in range(0, len(self.a_bc.indexes)):
                self.a[self.a_bc.indexes[counter]] = self.a_bc.accelerations[counter]()

    def single_tstep_integrate(self):
        self.a = -self.f_int / self.mass
        self.assemble_abcs() 
        if self.n == 0:
            self.v += 0.5 * self.a * self.dt
        else:
            self.v += self.a * self.dt
        self.assemble_vbcs(self.t + 0.5 * self.dt)
        self.u += self.v * self.dt
        if (self.formulation == "updated"):
            self.position += self.u # Updated Lagrangian
        self.n += 1
        self.compute_energy()
        self.t += self.dt
        self.f_int.fill(0)

    def compute_energy(self):
        self.timestamps.append(self.t)
        ke = 0
        ie = 0
        for i in range(self.n_nodes):
            ke += 0.5 * self.mass[i] * self.v[i]**2
        for j in range(self.n_elem):
            ie += ((0.5 * self.stress[j]**2) / self.E) * self.dx
        self.kinetic_energy.append(ke)
        self.internal_energy.append(ie)
        self.tot_energy.append(ke+ie)

## Output Plots for Simple Integrator Class

A Class is created to plot the FEM 1D Problems for both Total and Updated Lagrangian Formulations
This includes the creation of plots for:
 - Velocity throughout the domain (.gif)
 - Displacement throughout the domain (.gif)
 - Stress Distribution (.gif)
 - Conservation of Energy throughout the simulation (.png)

In [7]:
class Visualise_Monolithic:

    def __init__(self, totalLagrangian: SimpleIntegrator, updatedLagrangian: SimpleIntegrator):

        self.total = totalLagrangian
        self.updated = updatedLagrangian
        self.filenames_vel = []
        self.filenames_disp = []
        self.filenames_stress = []

    def plot_vel(self):
        self.filenames_vel.append(f'FEM1D_vel{self.total.n}{self.updated.n}.png')
        plt.style.use('ggplot')
        plt.plot(self.total.position, self.total.v, "--")
        plt.plot(self.updated.position, self.updated.v)
        plt.title(f"Graph of Velocity against Position for a Half Sine Excitation",fontsize=12)
        plt.xlabel("Domain Position (mm)")
        plt.ylabel("Velocity (mm/ms)")
        plt.legend([f"Total Lagrangian", "Updated Lagrangian"])
        plt.savefig(f'FEM1D_vel{self.total.n}{self.updated.n}.png')
        plt.close()

    def plot_disp(self):
        self.filenames_disp.append(f'FEM1D_disp{self.total.n}{self.updated.n}.png')
        plt.style.use('ggplot')
        plt.plot(self.total.position, self.total.u, "--")
        plt.plot(self.updated.position, self.updated.u)
        plt.title(f"Graph of Displacement against Position for a Half Sine Excitation",fontsize=12)
        plt.xlabel("Domain Position (mm)")
        plt.ylabel("Displacement (mm)")
        plt.legend([f"Total Lagrangian", "Updated Lagrangian"])
        plt.savefig(f'FEM1D_disp{self.total.n}{self.updated.n}.png')
        plt.close()

    def plot_stress(self):
        self.filenames_stress.append(f'FEM1D_stress{self.total.n}{self.updated.n}.png')
        plt.style.use('ggplot')
        plt.plot(self.total.midposition, self.total.stress, "--")
        plt.plot(self.updated.midposition, self.updated.stress)
        plt.title(f"Element Stress for a Half Sine Excitation",fontsize=12)
        plt.xlabel("Domain Position (mm)")
        plt.ylabel("Stress (GPa)")
        plt.legend([f"Total Lagrangian", "Updated Lagrangian"])
        plt.savefig(f'FEM1D_stress{self.total.n}{self.updated.n}.png')
        plt.close()

    def plot_energy(self):
        plt.style.use('ggplot')
        plt.locator_params(axis='both', nbins=4)
        plt.plot(self.total.timestamps, self.total.kinetic_energy, "--")
        plt.plot(self.updated.timestamps, self.updated.kinetic_energy)
        plt.plot(self.total.timestamps, self.total.internal_energy, "--")
        plt.plot(self.updated.timestamps, self.updated.internal_energy)
        plt.plot(self.total.timestamps, self.total.tot_energy, "--")
        plt.plot(self.updated.timestamps, self.updated.tot_energy)
        plt.title(f"Elastic Energies for a Half Sine Excitation",fontsize=12)
        plt.xlabel("Time (ms)")
        plt.ylabel("Energy (kN-mm)")
        plt.legend([f"Total Lagrangian KE", "Updated Lagrangian KE","Total Lagrangian IE", "Updated Lagrangian","Total Tot Lagrangian Total Energy", "Updated Tot Lagrangian Total Energy"])
        plt.savefig(f'FEM1D_enbal.png')
        plt.close()

    def create_gif(self, gif_name, filenames):
        with imageio.get_writer(gif_name, mode='I') as writer:
            for filename in filenames:
                image = imageio.imread(filename)
                writer.append_data(image)
        for filename in set(filenames):
            os.remove(filename)


## 1D Wave Propagation Example

In this numerical example we simulate the propagation of a half sinusoidal wave through an 
elastic domain

In [8]:
def monolithic():
    n_elem = 375
    E = 207
    rho = 7.83e-6
    L = 1
    propTime = 0.5*L * np.sqrt(rho / E)
    def vel(t): return vbc.velbc(t, L, E, rho)
    velboundaryConditions = vbc(list([0]), list([vel]))
    tot_formulation = "total"
    tot_bar = SimpleIntegrator(tot_formulation, E, rho, L, 1, n_elem, 2*propTime, velboundaryConditions, None, Co=1.0)
    bar = Visualise_Monolithic(tot_bar, tot_bar)
    while tot_bar.t <= tot_bar.tfinal:
        tot_bar.assemble_internal()
        tot_bar.single_tstep_integrate()
        bar.plot_vel()
        bar.plot_disp()
        bar.plot_stress()
    bar.plot_energy()
    # The evolution of Velocity, Displacement and Stress is plotted in the following gifs
    bar.create_gif('Updated_and_Total_1DFEM_vel.gif', bar.filenames_vel)
    bar.create_gif('Updated_and_Total_1DFEM_disp.gif', bar.filenames_disp)
    bar.create_gif('Updated_and_Total_1DFEM_stress.gif', bar.filenames_stress)

if __name__ == "__main__":
    monolithic()